## **Trabalho de Implementação** <br> COS360 - Otimização (2021.2)
### Aluno: <br> Pedro Boechat
<hr>

### • Importação dos módulos

In [2]:
using LinearAlgebra
using Printf

### • Problema de otimização
$$
    \begin{array}{rl}
        min & f(x) \\
        s.a. & x \in \Omega
    \end{array}
    \\
    \text{Dado que:}
    \\
    f(x_{1}, x_{2}) = x_{1}^{4} + x_{2}\sin(x_{1}) + x_{2}^{4}
    \\
    \Omega = \mathbb{R}^{2}
$$


### • Definições das funções
1. $f(x_{1}, x_{2}) = x_{1}^{4} + x_{2}\sin(x_{1}) + x_{2}^{4}$

In [3]:
function f(x)
    x[1]^4 + x[2]*sin(x[1]) + x[2]^4
end;

2. $\frac{d}{dx_{1}} f(x_{1}, x_{2}) = 4x_{1}^{3} + x_{2}\cos(x_{1})$

In [4]:
function dx₁_f(x)
    4.0*x[1]^3 + x[2]*cos(x[1])
end;

3. $\frac{d}{dx_{2}} f(x_{1}, x_{2}) = 4x_{2}^{3} + \sin(x_{1})$

In [5]:
function dx₂_f(x)
    4.0*x[2]^3 + sin(x[1])
end;

4. $\nabla f(x_{1}, x_{2}) = \begin{bmatrix} 4x_{1}^{3} + x_{2}\cos(x_{1}) \\ 4x_{2}^{3} + \sin(x_{1}) \end{bmatrix}$

In [6]:
function ∇f(x)
    [
        dx₁_f(x);
        dx₂_f(x)
    ]
end;

5. $\nabla^{2}f(x_{1}, x_{2}) = \begin{bmatrix} 12x_{1}^{2} - x_{2}\sin(x_{1}) & \cos(x_{1})\\ \cos(x_{1}) & 12x_{2}^{2} \end{bmatrix}$

In [7]:
function ∇²f(x)
    [
        12.0*(x[1]^2) - x[2]*sin(x[1])      cos(x[1]);
        cos(x[1])                           12.0*(x[2]^2)
    ]
end;

### • Análise da função
#### 1. Plots
##### a) $f(x_{1}, x_{2})$, $x_{1} \in [-1,1]$, $x_{2} \in [-1,1]$
![](./assets/graph1.jpg)
##### b) $f(x_{1}, x_{2})$, $x_{1} \in [-1,1]$, $x_{2} \in [-1,1]$ (visão frontal)
![](./assets/graph2.jpg)
##### c) Contorno de $f(x_{1}, x_{2})$, $x_{1} \in [-1,1]$, $x_{2} \in [-1,1]$
![](./assets/contour.jpg)
##### d) $f(x_{1}, x_{2})$, $x_{1} \in [-100,100]$, $x_{2} \in [-100,100]$
![](./assets/graph3.jpg)

#### 2. Pontos críticos
Um ponto $x^{*} \in \mathbb{R}^{n}$ que $\nabla f(x_{1}, x_{2}) = 0$ é dito ponto crítico da função $f$. Assim, temos que:<br>
![](./assets/critical.jpg)
$$
\begin{array}{l}
    p_{c_{1}} = (-0.476105,\ \ \ 0.485702)\\
    p_{c_{2}} = (\qquad\qquad 0,\qquad\qquad 0)\\
    p_{c_{3}} = (\ \ \ 0.476105, -0.485702)
\end{array}
$$
Para $p_{c_{1}}$ a hessiana será 
$\begin{bmatrix} 2.94272 & 0.888787 \\ 0.888787 & 2.83088 \end{bmatrix}$. Por ser definida positiva, 
podemos afimar que $p_{c_{1}}$ é um ponto de mínimo.<br><br>
Para $p_{c_{2}}$ a hessiana será 
$\begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$. Por ter determinante menor que zero, podemos afimar
que $p_{c_{2}}$ é um ponto de sela.<br><br>
Para $p_{c_{3}}$ a hessiana será 
$\begin{bmatrix} 2.94272 & 0.888787 \\ 0.888787 & 2.83088 \end{bmatrix}$. Por ser definida positiva, 
podemos afimar que $p_{c_{3}}$ é um ponto de mínimo.

#### 3. Convexidade
##### a) Definição de função convexa
Seja $C \subset \mathbb{R}^{n}$ um conjunto convexo. Dizemos que a função 
$f: \mathbb{R}^{n}\rightarrow\mathbb{R}$ é convexa em $C$ quando 
$f((1-t)x + ty) \le (1 - t)f(x) + tf(y),\ \forall x,y \in C,\ t \in [0,1]$.

##### b) Teorema 19 - Slide "Convexidade"
Sejam $f: \mathbb{R}^{n}\rightarrow\mathbb{R}$ uma função duas vezes diferenciável e 
$C \subset \mathbb{R}^{n}$ convexo.<br>

$I.$ Se $\nabla^{2}f(x) \ge 0$, para todo $x \in C$, então $f$ é convexa em $C$.<br>
$II.$ Se $f$ é convexa em $C$ e $\mathsf{int}(C) \neq \{\emptyset\}$, então $\nabla^{2}f(x) \ge 0$, 
para todo $x \in C$.

##### c) Análise de convexidade
$f(x_{1}, x_{2})$ é convexa para todos $x_{1}, x_{2} \in \mathbb{R}^{2}$ exceto na região próxima
aos pontos críticos. Como a função tende a $x_{1}^{4} + x_{2}x_{1} + x_{2}^{4}$ no infinito, 
podemos afirmar que é convexa.<br>

### • Implementação dos métodos
#### 1. Busca de Armijo
A Busca de Armijo procura uma boa redução da função ao longo da direção, sem tentar minimizá-la.<br>
`x` é um ponto;<br>
`d` é a direção de decida;<br>
`η` é o erro;<br>
`γ` é o fator de correção;<br>
`t₀` é o tamanho inicial do passo.<br>

In [8]:
function armijo(x, d, η, γ, t₀)
    k = 0
    t = t₀
    while f(x + t*d) > f(x)+η*t*(transpose(∇f(x))*d)
        t = γ*t
        k += 1
    end
    [t, k]
end;

#### 2. Método do gradiente
O método do gradiente faz a busca na direção oposta ao vetor gradiente da função objetivo no ponto
corrente.<br>
`x⁰` é o ponto inicial;<br>
`η` é o erro para a Busca de Armijo;<br>
`γ` é o fator de correção para a Busca de Armijo;<br>
`t₀` é o tamanho inicial do passo para a Busca de Armijo;<br>
`tol` é a tolerância do critério de parada;<br>
`n_iter` é o número máximo de iterações.<br>

In [9]:
function gradiente(x⁰, η, γ; t₀=1, tol=10^-6, n_iter=10^3)
    k = 0
    k_arm = 0
    xᵏ = x⁰
    while (norm(∇f(xᵏ)) > tol) & (k < n_iter)
        dᵏ = -∇f(xᵏ)
        resultado_armijo = armijo(xᵏ, dᵏ, η, γ, t₀)
        tₖ = resultado_armijo[1]
        k_arm += resultado_armijo[2]
        xᵏ += tₖ*dᵏ
        k += 1
    end
    if k == n_iter
        println("Limite de iterações alcançado.")
        println("Iter. Armijo = ", k_arm)
        return xᵏ
    end
    println("Convergiu após ", k, " iterações.")
    println("Iter. Armijo = ", k_arm)
    return xᵏ
end;

#### 3. Método de Newton
O método de Newton utiliza um polinômio de Taylor de primeira ordem para aproximar o sistema.<br>
`x⁰` é o ponto inicial;<br>
`η` é o erro para a Busca de Armijo;<br>
`γ` é o fator de correção para a Busca de Armijo;<br>
`t₀` é o tamanho inicial do passo para a Busca de Armijo;<br>
`puro` é se será utilizado o método puro (tₖ=1);<br>
`tol` é a tolerância do critério de parada;<br>
`n_iter` é o número máximo de iterações.<br>

In [10]:
function newton(x⁰, η, γ; t₀=1, puro=false, tol=10^-6, n_iter=10^3)
    k = 0
    k_arm = 0
    xᵏ = x⁰
    while (norm(∇f(xᵏ)) > tol) & (k < n_iter)
        dᵏ = -inv(∇²f(xᵏ))*∇f(xᵏ)
        if (puro)
            tₖ = 1
        else
            resultado_armijo = armijo(xᵏ, dᵏ, η, γ, t₀)
            tₖ = resultado_armijo[1]
            k_arm += resultado_armijo[2]
        end
        xᵏ += tₖ*dᵏ
        k += 1
    end
    if k == n_iter
        println("Limite de iterações alcançado.")
        println("Iter. Armijo = ", k_arm)
        return xᵏ
    end
    println("Convergiu após ", k, " iterações.")
    println("Iter. Armijo = ", k_arm)
    return xᵏ
end;

#### 4. Método quase-Newton
O método quase-Newton constrói aproximações para a Hessiana (usando BFGS ou DFP) da função objetivo 
e busca na direção de $-H_{k}\nabla f(x^{k})$.<br>
`x⁰` é o ponto inicial;<br>
`η` é o erro para a Busca de Armijo;<br>
`γ` é o fator de correção para a Busca de Armijo;<br>
`t₀` é o tamanho inicial do passo para a Busca de Armijo;<br>
`use_bfgs` é se será utilizado o BFGS, caso contrário será utilizado DFP;<br>
`tol` é a tolerância do critério de parada;<br>
`n_iter` é o número máximo de iterações.<br>

In [11]:
function bfgs(Hₖ, pᵏ, qᵏ)
    pᵏ = reshape(pᵏ, (2,1))
    qᵏ = reshape(qᵏ, (2,1))
    pᵏᵀ = transpose(pᵏ)
    qᵏᵀ = transpose(qᵏ)
    Hₖ₊₁ = Hₖ + (
        1 .+ (
            (qᵏᵀ*Hₖ*qᵏ) ./ (pᵏᵀ*qᵏ)
        )
    ) .* (
        (pᵏ*pᵏᵀ) ./ (pᵏᵀ*qᵏ)
    ) - (
        ((pᵏ*qᵏᵀ*Hₖ)+(Hₖ*qᵏ*pᵏᵀ)) ./ (pᵏᵀ*qᵏ)
    )
    return Hₖ₊₁
end

function dfp(Hₖ, pᵏ, qᵏ)
    pᵏ = reshape(pᵏ, (2,1))
    qᵏ = reshape(qᵏ, (2,1))
    pᵏᵀ = transpose(pᵏ)
    qᵏᵀ = transpose(qᵏ)
    Hₖ₊₁ = Hₖ + (
        (pᵏ*pᵏᵀ) ./ (pᵏᵀ*qᵏ)
    ) - (
        (Hₖ*qᵏ*qᵏᵀ*Hₖ) ./ (qᵏᵀ*Hₖ*qᵏ)
    )
    return Hₖ₊₁
end

function quase_newton(x⁰, η, γ; H₀=[1 0;0 1], t₀=1, use_bfgs=true, tol=10^-6, n_iter=10^3)
    if (use_bfgs)
        println("Usando BFGS")
        atualiza_H = bfgs
    else
        println("Usando DFP")
        atualiza_H = dfp
    end
    k = 0
    k_arm = 0
    xᵏ = x⁰
    Hₖ = H₀
    pᵏ = x⁰
    qᵏ = ∇f(x⁰)
    while (norm(∇f(xᵏ)) > tol) & (k < n_iter)
        x⁰ = xᵏ
        dᵏ = -Hₖ*∇f(xᵏ)
        resultado_armijo = armijo(xᵏ, dᵏ, η, γ, t₀)
        tₖ = resultado_armijo[1]
        k_arm += resultado_armijo[2]
        xᵏ += tₖ*dᵏ
        pᵏ = xᵏ - x⁰
        qᵏ = ∇f(xᵏ) - ∇f(x⁰)
        Hₖ = atualiza_H(Hₖ, pᵏ, qᵏ)
        k += 1
    end
    if k == n_iter
        println("Limite de iterações alcançado.")
        println("Iter. Armijo = ", k_arm)
        return xᵏ
    end
    println("Convergiu após ", k, " iterações.")
    println("Iter. Armijo = ", k_arm)
    return xᵏ
end;

### • Execução dos métodos
#### 0. Preparativos
##### a) Lista de vetores $X^{0}$

In [12]:
𝑋⁰ = [
    [-100;200],
    [-2;2],
    [-0.1;-0.1],
    [0;0],
    [0.1;0.1],
    [2;-2],
    [100;-200]
];

##### b) Teste para $\eta$ e $\gamma$

In [13]:
for method in [gradiente, newton, quase_newton]
    println(method, "\n----------------------------\n")
    for eta in [0.1, 0.25, 0.5, 0.75, 0.9]
        for gamma in [0.1, 0.2, 0.4, 0.6, 0.8]
            println("η = ", eta, "; γ = ", gamma)
            resultado = method([-100;200], eta, gamma)
            println(resultado, "\n")
        end
    end
end

gradiente
----------------------------

η = 0.1; γ = 0.1
Convergiu após 74 iterações.
Iter. Armijo = 86.0
[-0.47610443051301554, 0.48570157744702813]

η = 0.1; γ = 0.2
Convergiu após 33 iterações.
Iter. Armijo = 61.0
[-0.476104492744593, 0.48570164371719177]

η = 0.1; γ = 0.4
Convergiu após 24 iterações.
Iter. Armijo = 56.0
[-0.4761045934979279, 0.48570201014382175]

η = 0.1; γ = 0.6
Convergiu após 22 iterações.
Iter. Armijo = 91.0
[-0.47610459964662794, 0.4857020798763354]

η = 0.1; γ = 0.8
Convergiu após 40 iterações.
Iter. Armijo = 351.0
[-0.4761045749165639, 0.485702027593217]

η = 0.25; γ = 0.1
Convergiu após 70 iterações.
Iter. Armijo = 102.0
[-0.476104418515813, 0.48570156467100845]

η = 0.25; γ = 0.2
Convergiu após 33 iterações.
Iter. Armijo = 61.0
[-0.476104492744593, 0.48570164371719177]

η = 0.25; γ = 0.4
Convergiu após 22 iterações.
Iter. Armijo = 75.0
[-0.4761044929288658, 0.48570201153591863]

η = 0.25; γ = 0.6
Convergiu após 18 iterações.
Iter. Armijo = 107.0
[-0.4761045

#### 1. Método do gradiente

In [14]:
println("Método do gradiente\n----------------------------------------")
for x in 𝑋⁰
    resultado_gradiente = gradiente(x, 0.25, 0.8)
    @printf "𝑥⁰ = %s\n" x
    @printf "𝑥₁ = %.3e, 𝑥₂ = %.3e\n" resultado_gradiente[1] resultado_gradiente[2]
    @printf "𝑓(𝑥₁,𝑥₂) = %.3e, error = %.3e\n" f(resultado_gradiente) norm(∇f(resultado_gradiente))
    println("----------------------------------------")
end

Método do gradiente
----------------------------------------
Convergiu após 15 iterações.
Iter. Armijo = 197.0
𝑥⁰ = [-100.0, 200.0]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 9.146e-07
----------------------------------------
Convergiu após 14 iterações.
Iter. Armijo = 67.0
𝑥⁰ = [-2.0, 2.0]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 3.574e-07
----------------------------------------
Convergiu após 22 iterações.
Iter. Armijo = 40.0
𝑥⁰ = [-0.1, -0.1]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 3.110e-07
----------------------------------------
Convergiu após 0 iterações.
Iter. Armijo = 0
𝑥⁰ = [0.0, 0.0]
𝑥₁ = 0.000e+00, 𝑥₂ = 0.000e+00
𝑓(𝑥₁,𝑥₂) = 0.000e+00, error = 0.000e+00
----------------------------------------
Convergiu após 22 iterações.
Iter. Armijo = 40.0
𝑥⁰ = [0.1, 0.1]
𝑥₁ = 4.761e-01, 𝑥₂ = -4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 3.110e-07
----------------------------------------
Convergiu após 14 iterações.
Iter. Armijo

In [15]:
println("Método de Newton\n----------------------------------------")
for x in 𝑋⁰
    resultado_newton = newton(x, 0.25, 0.8)
    @printf "𝑥⁰ = %s\n" x
    @printf "𝑥₁ = %.3e, 𝑥₂ = %.3e\n" resultado_newton[1] resultado_newton[2]
    @printf "𝑓(𝑥₁,𝑥₂) = %.3e, error = %.3e\n" f(resultado_newton) norm(∇f(resultado_newton))
    println("----------------------------------------")
end

Método de Newton
----------------------------------------
Convergiu após 19 iterações.
Iter. Armijo = 0
𝑥⁰ = [-100.0, 200.0]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 1.366e-12
----------------------------------------
Convergiu após 7 iterações.
Iter. Armijo = 0
𝑥⁰ = [-2.0, 2.0]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 8.203e-07
----------------------------------------
Convergiu após 3 iterações.
Iter. Armijo = 0
𝑥⁰ = [-0.1, -0.1]
𝑥₁ = -1.059e-16, 𝑥₂ = -4.305e-17
𝑓(𝑥₁,𝑥₂) = 4.559e-33, error = 1.143e-16
----------------------------------------
Convergiu após 0 iterações.
Iter. Armijo = 0
𝑥⁰ = [0.0, 0.0]
𝑥₁ = 0.000e+00, 𝑥₂ = 0.000e+00
𝑓(𝑥₁,𝑥₂) = 0.000e+00, error = 0.000e+00
----------------------------------------
Convergiu após 3 iterações.
Iter. Armijo = 0
𝑥⁰ = [0.1, 0.1]
𝑥₁ = 1.059e-16, 𝑥₂ = 4.305e-17
𝑓(𝑥₁,𝑥₂) = 4.559e-33, error = 1.143e-16
----------------------------------------
Convergiu após 7 iterações.
Iter. Armijo = 0
𝑥⁰ = [2.0, -2.0]


In [16]:
println("Método quase-Newton\n----------------------------------------")
for method in [true, false]
    for x in 𝑋⁰
        resultado_quase_newton = quase_newton(x, 0.25, 0.8, use_bfgs=method, tol=10^-3)
        @printf "𝑥⁰ = %s\n" x
        @printf "𝑥₁ = %.3e, 𝑥₂ = %.3e\n" resultado_quase_newton[1] resultado_quase_newton[2]
        @printf "𝑓(𝑥₁,𝑥₂) = %.3e, error = %.3e\n" f(resultado_quase_newton) norm(∇f(resultado_quase_newton))
        println("----------------------------------------")
    end
end

Método quase-Newton
----------------------------------------
Usando BFGS
Convergiu após 35 iterações.
Iter. Armijo = 101.0
𝑥⁰ = [-100.0, 200.0]
𝑥₁ = 4.761e-01, 𝑥₂ = -4.856e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 3.028e-04
----------------------------------------
Usando BFGS
Convergiu após 10 iterações.
Iter. Armijo = 15.0
𝑥⁰ = [-2.0, 2.0]
𝑥₁ = -4.761e-01, 𝑥₂ = 4.857e-01
𝑓(𝑥₁,𝑥₂) = -1.156e-01, error = 2.035e-05
----------------------------------------
Usando BFGS
Convergiu após 2 iterações.
Iter. Armijo = 0
𝑥⁰ = [-0.1, -0.1]
𝑥₁ = -2.019e-04, 𝑥₂ = 4.526e-04
𝑓(𝑥₁,𝑥₂) = -9.140e-08, error = 4.956e-04
----------------------------------------
Usando BFGS
Convergiu após 0 iterações.
Iter. Armijo = 0
𝑥⁰ = [0.0, 0.0]
𝑥₁ = 0.000e+00, 𝑥₂ = 0.000e+00
𝑓(𝑥₁,𝑥₂) = 0.000e+00, error = 0.000e+00
----------------------------------------
Usando BFGS
Convergiu após 2 iterações.
Iter. Armijo = 0
𝑥⁰ = [0.1, 0.1]
𝑥₁ = 2.019e-04, 𝑥₂ = -4.526e-04
𝑓(𝑥₁,𝑥₂) = -9.140e-08, error = 4.956e-04
------------------------------

### • Resultados
#### 1. Método do gradiente
|      $X^{0}$     | Iter. | Call. Armijo |        Opt. Point        | Opt. Value |   Error   |
|:----------------:|:-----:|:------------:|:------------------------:|:----------:|:---------:|
| (-100.0,  200.0) | 15    | 197          | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 9.146e-07 |
| (  -2.0,    2.0) | 14    | 67           | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 3.574e-07 |
| (  -0.1,   -0.1) | 22    | 40           | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 3.110e-07 |
| (   0.0,    0.0) | 0     | 0            | (0, 0)                   | 0          | 0         |
| (   0.1,    0.1) | 22    | 40           | (4.761e-01, -4.857e-01)  | -1.156e-01 | 3.110e-07 |
| (   2.0,   -2.0) | 14    | 67           | (4.761e-01, -4.857e-01)  | -1.156e-01 | 3.574e-07 |
| ( 100.0, -200.0) | 15    | 197          | (4.761e-01, -4.857e-01)  | -1.156e-01 | 9.146e-07 |

#### 2. Método de Newton
|      $X^{0}$     | Iter. | Call. Armijo |        Opt. Point        | Opt. Value |   Error   |
|:----------------:|:-----:|:------------:|:------------------------:|:----------:|:---------:|
| (-100.0,  200.0) | 19    | 0            | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 1.366e-12 |
| (  -2.0,    2.0) | 7     | 0            | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 8.203e-07 |
| (  -0.1,   -0.1) | 3     | 0            | (-1.059e-16, -4.305e-17) | -1.156e-01 | 1.143e-16 |
| (   0.0,    0.0) | 0     | 0            | (0, 0)                   | 0          | 0         |
| (   0.1,    0.1) | 3     | 0            | (1.059e-16, 4.305e-17)   | -1.156e-01 | 1.143e-16 |
| (   2.0,   -2.0) | 7     | 0            | (4.761e-01, -4.857e-01)  | -1.156e-01 | 8.203e-07 |
| ( 100.0, -200.0) | 19    | 0            | (4.761e-01, -4.857e-01)  | -1.156e-01 | 1.366e-12 |

#### 3. Método quase-Newton
##### a) BFGS
|      $X^{0}$     | Iter. | Call. Armijo |        Opt. Point        | Opt. Value |   Error   |
|:----------------:|:-----:|:------------:|:------------------------:|:----------:|:---------:|
| (-100.0,  200.0) | 35    | 101          | (4.761e-01, -4.857e-01)  | -1.156e-01 | 3.028e-04 |
| (  -2.0,    2.0) | 10    | 15           | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 2.035e-05 |
| (  -0.1,   -0.1) | 2     | 0            | (-2.019e-04, 4.526e-04)  | -9.140e-08 | 4.956e-04 |
| (   0.0,    0.0) | 0     | 0            | (0, 0)                   | 0          | 0         |
| (   0.1,    0.1) | 2     | 0            | (2.019e-04, -4.526e-04)  | -9.140e-08 | 4.956e-04 |
| (   2.0,   -2.0) | 10    | 15           | (4.761e-01, -4.857e-01)  | -1.156e-01 | 2.035e-05 |
| ( 100.0, -200.0) | 35    | 101          | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 3.028e-04 |
##### b) DFP
|      $X^{0}$     | Iter. | Call. Armijo |        Opt. Point        | Opt. Value |   Error   |
|:----------------:|:-----:|:------------:|:------------------------:|:----------:|:---------:|
| (-100.0,  200.0) | 114   | 127          | (4.761e-01, -4.857e-01)  | -1.156e-01 | 2.961e-04 |
| (  -2.0,    2.0) | 39    | 21           | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 8.333e-04 |
| (  -0.1,   -0.1) | 2     | 0            | (-2.019e-04, 4.526e-04)  | -9.140e-08 | 4.956e-04 |
| (   0.0,    0.0) | 0     | 0            | (0, 0)                   | 0          | 0         |
| (   0.1,    0.1) | 2     | 0            | (2.019e-04, -4.526e-04)  | -9.140e-08 | 4.956e-04 |
| (   2.0,   -2.0) | 39    | 21           | (4.761e-01, -4.857e-01)  | -1.156e-01 | 8.333e-04 |
| ( 100.0, -200.0) | 114   | 127          | (-4.761e-01, 4.857e-01)  | -1.156e-01 | 2.961e-04 |